# Klue/bert-base를 Augmented Data로 학습시키기

Augmentation을 사용하여 데이터의 갯수를 증가시킨 데이터로 모델을 학습시킵니다.   
증강된 데이터는 원본 데이터와 비슷한 정보를 가지고 있기 때문에, 학습 데이터와 검증 데이터가 섞인다면 정확한 성능 측정이 되지 않을 수 있습니다.  
따라서, 학습 데이터와 검증 데이터가 섞이지 않도록 주의해야합니다.  
이번 노트북에서는 학습 데이터와 검증 데이터를 나눈뒤, 학습 데이터에 데이터 증강 기법을 사용하고 모델을 훈련시킵니다. 



필요한 라이브러리들을 불러오고, 학습에 사용되는 GPU를 확인합니다

In [1]:
import random
from tqdm.notebook import tqdm, tnrange
import os

import numpy as np
import pandas as pd
import datasets
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

from transformers import AdamW
from transformers import get_linear_schedule_with_warmup

from sklearn.model_selection import train_test_split

import torch
from torch import nn
from torch.utils.data import Dataset,TensorDataset, DataLoader, RandomSampler

from sklearn.metrics import accuracy_score

if torch.cuda.is_available():
    print("사용가능한 GPU수 : ",torch.cuda.device_count())
    device = torch.device("cuda")
else:
    print("CPU 사용")
    device = torch.device("cpu")

사용가능한 GPU수 :  1


Reproduction을 위한 Seed 고정  
출처 : https://dacon.io/codeshare/2363?dtype=vote&s_id=0

In [2]:
RANDOM_SEED = 42

def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore
    
seed_everything(RANDOM_SEED)

이번 노트북에서는 klue/bert-base 모델을 사용합니다. 

In [3]:
model_checkpoint = "klue/bert-base"
batch_size = 32
task = "nli"
MODEL_P = "models/klue-bert-base-augmented.pth"

huggingface 에서 tokenizer를 불러옵니다.

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

huggingface 에서 model를 불러옵니다.

In [5]:
num_labels = 7
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint,num_labels=num_labels)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

dataset을 가져옵니다. 

In [6]:
dataset = pd.read_csv("data/train_data.csv",index_col=False)
test = pd.read_csv("data/test_data.csv",index_col=False)

train 데이터와 test 데이터를 나눠줍니다. 

In [7]:
dataset_train, dataset_val = train_test_split(dataset,test_size = 0.2,random_state = RANDOM_SEED)

이번 노트북에서는 RD(Random Deletion)와 RS(Random Swap) 기법을 사용하여 augmentation을 진행합니다.   

문장을 토큰화 해주도록 하겠습니다. 

In [8]:
dataset_train["tokenized"] = [tokenizer.tokenize(sentence) for sentence in dataset_train["title"]]

d:\workspace\dacon_news_topic_clasiification\venv\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


토큰화가 잘 되었는지 확인합니다.

In [9]:
dataset_train.head()

,index,title,topic_idx,tokenized
4824,4824,KT 화상회의 가능한 기업용 전화 출시,1,"[KT, 화상, ##회의, 가능, ##한, 기업, ##용, 전화, 출시]"
21714,21714,문 대통령 해리 해리스 주한 미국대사와 함께,6,"[문, 대통령, 해리, 해리, ##스, 주한, 미국, ##대사, ##와, 함께]"
29145,29145,김기식 사퇴 찬성 51%…文대통령 지지율 66.2%로 하락리얼미터,6,"[김기, ##식, 사퇴, 찬성, 51, %, …, 文, 대통령, 지지율, 66, ...."
18342,18342,민주콩고서 수백명 에볼라 사태 끝내자 거리행진,4,"[민주, ##콩, ##고, ##서, 수백, ##명, 에볼라, 사태, 끝내, ##자,..."
9010,9010,충남 올겨울 첫 한파주의보…눈 최대 15㎝ 쌓일 듯,3,"[충남, 올겨울, 첫, 한파, ##주의, ##보, …, 눈, 최대, 15, ##㎝,..."


RD를 적용하기 위해 random_deletion 함수를 정의합니다. 

In [10]:
def random_deletion(words, p):
    if len(words) == 1:
        return words

    new_words = []
    for word in words:
        r = random.uniform(0, 1)
        if r > p:
            new_words.append(word)

    if len(new_words) == 0:
        rand_int = random.randint(0, len(words)-1)
        return [words[rand_int]]

    return new_words

토큰화된 문장에 random_deletion함수를 적용합니다.

In [11]:
rd = [random_deletion(tokenized,0.2) for tokenized in dataset_train["tokenized"]]

토큰이 랜덤으로 삭제된 것을 볼 수 있습니다.

In [12]:
dataset_train["tokenized"].iloc[0] , rd[0]

(['KT', '화상', '##회의', '가능', '##한', '기업', '##용', '전화', '출시'],
 ['KT', '##회의', '가능', '##한', '기업', '##용', '출시'])

랜덤으로 삭제된 데이터를 DataFrame의 형태로 만들어줍니다. 나중에 dataset_train과 합칠 것입니다.

In [13]:
rd_augmented_dataset = pd.DataFrame({"tokenized" : rd, "topic_idx": dataset_train["topic_idx"]})

이번엔 RS를 적용하기 위해 random_swap 함수를 정의합니다.

In [14]:
def random_swap(words, n):
    new_words = words.copy()
    for _ in range(n):
        new_words = swap_word(new_words)

    return new_words

def swap_word(new_words):
    random_idx_1 = random.randint(0, len(new_words)-1)
    random_idx_2 = random_idx_1
    counter = 0

    while random_idx_2 == random_idx_1:
        random_idx_2 = random.randint(0, len(new_words)-1)
        counter += 1
        if counter > 3:
            return new_words

    new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
    return new_words

토큰화된 문장에 random_swap함수를 적용합니다.

In [15]:
rs = [random_swap(tokenized,2) for tokenized in dataset_train["tokenized"]]

토큰의 위치가 임의로 바뀐 것을 볼 수 있습니다. 

In [16]:
dataset_train["tokenized"].iloc[0] , rs[0]

(['KT', '화상', '##회의', '가능', '##한', '기업', '##용', '전화', '출시'],
 ['KT', '화상', '##용', '##한', '가능', '기업', '##회의', '전화', '출시'])

DataFrame의 형태로 만들어줍니다.  

In [17]:
rs_augmented_dataset = pd.DataFrame({"tokenized" : rs, "topic_idx": dataset_train["topic_idx"]})

RD, RS가 적용된 데이터셋과 학습 데이터셋을 합치고, 순서를 섞어줍니다. 

In [18]:
dataset_train = pd.concat([dataset_train,rd_augmented_dataset,rs_augmented_dataset])

In [19]:
dataset_train = dataset_train.sample(frac=1, random_state=RANDOM_SEED).reset_index(drop=True)
dataset_train["tokenized"][:10]

0    [미, ##중, ##갈, ##등, 한복, ##판, ##에, 선, 中, 화웨이, 멍,...
1    [SK, 박종훈, 한, 이닝, ##에, 몸, ##에, 맞, ##는, 공, 3, ##...
2    [김광석, 뮤지컬, 시름, 김광석, 없, ##에, 이유, ##는, …, 저작권, 시...
3             [그래픽, 국내, 시장, ##별, 평균, 주식, ##보, ##유, 기간]
4                         [박성, ##제, MBC, 신임, 보도, ##국장]
5    [##배, ##구, V, ##리그, 올스타, ##전, 팬, ##표, 7, ##일, 시작]
6        [건국, ##언, ##론, ##인, ##상, 이동, ##희, ·, 심재, ##윤]
7    [북한, 고위급, 인사, 베이징, 도착, …, 귀, ##빈, 차량, ##으로, 이동...
8    [38, ##점, 폭발, ##한, 거포, 디, ##우, ##프, 인삼, ##공사, ...
9    [주말, N, 여행, 제주, ##권, 인류, ##무, ##형, ##문화, ##유산,...
Name: tokenized, dtype: object

데이터의 수가 늘어났습니다. 

In [20]:
len(dataset_train)

109569

학습 데이터들은 토큰화가 된 상태이고, 테스트 데이터들은 문장의 형태로 서로 다릅니다.  
그래서 학습에 필요한 데이터셋을 만드는 방식과 테스트에 필요한 데이터를 만드는 방식이 다릅니다.  
따라서 서로 다른 데이터셋으로 정의해줍니다. 

학습데이터의 경우 토큰화가 되어있기 때문에 `tokenizer`의 `tokenize`함수를 사용할 수 없습니다. 따라서 `convert_tokens_to_ids`함수를 사용하여 토큰을 input_ids로 바꿔줍니다. 

In [21]:
print(dataset_train["tokenized"].iloc[0])
tokenizer.convert_tokens_to_ids(dataset_train["tokenized"].iloc[0])

['미', '##중', '##갈', '##등', '한복', '##판', '##에', '선', '中', '화웨이', '멍', '##완', '##저우', '결백', '주장']


[1107,
 2284,
 2577,
 2491,
 11641,
 2025,
 2170,
 1261,
 220,
 21157,
 1064,
 2365,
 11110,
 29282,
 3831]

이때, [CLS]나 [SEP]과 같은 special token이 자동으로 추가되지 않습니다. 따라서 수동으로 token을 추가해주어야 합니다.   
tokenizer의 cls 토큰하고 sep 토큰의 ids를 확인합니다. 

In [22]:
tokenizer.convert_tokens_to_ids(tokenizer.cls_token),tokenizer.convert_tokens_to_ids(tokenizer.sep_token)

(2, 3)

`convert_tokens_to_ids` 함수를 실행한 뒤, list의 맨 앞에 [CLS]토큰의 ids를, 맨 뒤에는 [SEP]토큰의 ids를 추가해줍니다.   
적절한 attention_masks와 `__getitem__`함수, `__len__`함수를 만들어줍니다. 

In [23]:
class TrainDataset(Dataset):
    def __init__(self, dataset, sent_key, label_key, bert_tokenizer):
        
        self.sentences = [ bert_tokenizer.convert_tokens_to_ids(i) for i in dataset[sent_key] ]
        for idx in range(len(self.sentences)):
            self.sentences[idx].insert(0,2)
            self.sentences[idx].append(3)
        
        self.attention_masks = [ [1 for i in range(len(sentence))] for sentence in self.sentences ]
        self.labels = [np.int64(i) for i in dataset[label_key]]


    def __getitem__(self, i):
        return {'input_ids':self.sentences[i],'attention_mask': self.attention_masks[i],'label': self.labels[i]}


    def __len__(self):
        return (len(self.labels))

테스트 데이터셋은 `tokenizer`의 `tokenize`함수를 사용합니다.  
검증 데이터에는 라벨이 있지만, 테스트 데이터에는 라벨이 없기 때문에 테스트 데이터의 경우에는 라벨을 0으로 초기화해줍니다. 

In [24]:
class TestDataset(Dataset):
    def __init__(self, dataset, sent_key, label_key, bert_tokenizer):
        
        self.sentences = [ bert_tokenizer(i,truncation=True,return_token_type_ids=False) for i in dataset[sent_key] ]
        
        if not label_key == None:
            self.mode = "train"
        else:
            self.mode = "test"
            
        if self.mode == "train":
            self.labels = [np.int64(i) for i in dataset[label_key]]
        else:
            self.labels = [np.int64(0) for i in dataset[sent_key]]

    def __getitem__(self, i):
        if self.mode == "train":
            self.sentences[i]["label"] = self.labels[i]
            return self.sentences[i]
        else:
            return self.sentences[i]

    def __len__(self):
        return (len(self.labels))


데이터셋을 만들어줍니다.

In [25]:
data_train = TrainDataset(dataset_train, "tokenized", "topic_idx", tokenizer)
data_val = TestDataset(dataset_val, "title", "topic_idx", tokenizer)
data_test = TestDataset(test, "title", None, tokenizer)

In [26]:
data_val[3]

{'input_ids': [2, 4816, 2223, 7270, 7270, 2882, 2292, 9608, 17797, 2361, 2063, 7355, 2170, 8172, 27854, 3], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'label': 1}

In [27]:
data_train[0]

{'input_ids': [2,
  1107,
  2284,
  2577,
  2491,
  11641,
  2025,
  2170,
  1261,
  220,
  21157,
  1064,
  2365,
  11110,
  29282,
  3831,
  3],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'label': 4}

trainer에서 사용할 metric을 정의해줍니다. 

In [28]:
metric = load_metric("glue", "qnli")

In [29]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

trainer에 들어갈 Arguments를 정의해줍니다.  
증강된 데이터를 사용하면 한 epoch이 원래 데이터의 3배이기 때문에 epoch마다 평가를 진행할 경우 overfitting이 발생할 수 있습니다.   
따라서 좀더 작은 단위인 step단위로 평가를 진행하기 위해 `evaluation_strategy`를 `epoch`이 아닌 `steps`로 사용했습니다.   

In [30]:
metric_name = "accuracy"

args = TrainingArguments(
    MODEL_P,
    evaluation_strategy="steps",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=2,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

trainer를 정의하고 학습을 진행합니다. 

In [31]:
trainer = Trainer(
    model,
    args,
    train_dataset=data_train,
    eval_dataset=data_val,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [32]:
trainer.train()

***** Running training *****
  Num examples = 109569
  Num Epochs = 2
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 6850


Step,Training Loss,Validation Loss,Accuracy
500,0.529200,0.372666,0.877889
1000,0.380700,0.342845,0.882160
1500,0.346700,0.326366,0.888293
2000,0.314000,0.333488,0.887745
2500,0.300900,0.333628,0.885774
3000,0.282500,0.328829,0.888950
3500,0.254100,0.343085,0.891578
4000,0.197700,0.360881,0.891031
4500,0.177400,0.363702,0.891140
5000,0.169300,0.374855,0.890045


***** Running Evaluation *****
  Num examples = 9131
  Batch size = 32
Saving model checkpoint to models/klue-bert-base-augmented.pth\checkpoint-500
Configuration saved in models/klue-bert-base-augmented.pth\checkpoint-500\config.json
Model weights saved in models/klue-bert-base-augmented.pth\checkpoint-500\pytorch_model.bin
tokenizer config file saved in models/klue-bert-base-augmented.pth\checkpoint-500\tokenizer_config.json
Special tokens file saved in models/klue-bert-base-augmented.pth\checkpoint-500\special_tokens_map.json
***** Running Evaluation *****
  Num examples = 9131
  Batch size = 32
Saving model checkpoint to models/klue-bert-base-augmented.pth\checkpoint-1000
Configuration saved in models/klue-bert-base-augmented.pth\checkpoint-1000\config.json
Model weights saved in models/klue-bert-base-augmented.pth\checkpoint-1000\pytorch_model.bin
tokenizer config file saved in models/klue-bert-base-augmented.pth\checkpoint-1000\tokenizer_config.json
Special tokens file saved in m

TrainOutput(global_step=6850, training_loss=0.2580998591959041, metrics={'train_runtime': 806.1617, 'train_samples_per_second': 271.829, 'train_steps_per_second': 8.497, 'total_flos': 3109487374633926.0, 'train_loss': 0.2580998591959041, 'epoch': 2.0})

학습이 끝난 뒤, 평가를 통해 결과를 확인합니다.

In [33]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 9131
  Batch size = 32


{'eval_loss': 0.34308499097824097,
 'eval_accuracy': 0.8915781404008323,
 'eval_runtime': 7.7037,
 'eval_samples_per_second': 1185.281,
 'eval_steps_per_second': 37.125,
 'epoch': 2.0}

`predict`함수를 사용하여 테스트 데이터셋에 대한 예측을 진행합니다. 

In [34]:
pred = trainer.predict(data_test)

***** Running Prediction *****
  Num examples = 9131
  Batch size = 32


결과를 csv파일로 저장하고 제출합니다. 

In [35]:
pred = pred[0]

In [36]:
pred = np.argmax(pred,1)

In [37]:
submission = pd.read_csv('data/sample_submission.csv')
submission['topic_idx'] = pred
submission.to_csv("results/klue-bert-base-simple-rd-rs.csv",index=False)